In [1]:
%run feature_extraction.ipynb

/home/iten/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


3.246347453321581e-06
6.951395840284729e-06


In [2]:
import tqdm
import torch
import numpy as np
import pandas as pd
from numpy import mean, std
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score, GridSearchCV

In [3]:
def edit_categories(x):
    if x == 'info_news':
        return 0
    elif x == 'celebrity':
        return 1
    elif x == 'plan':
        return 2
    elif x == 'requests':
        return 3
    elif x == 'rumors':
        return 4
    elif x == 'advice':
        return 5
    elif x == 'restrictions':
        return 6
    elif x == 'personal':
        return 7
    elif x == 'unrelated':
        return 8
    elif x == 'others':
        return 9
    else:
        return -1

In [4]:
train_1=pd.read_pickle('output/train_3_original.pkl')
dev_1=pd.read_pickle('output/dev_1_original.pkl')

In [5]:
sentences_train, stance_train, category_train = train_1.loc[:,"tokens"],train_1.loc[:,"stance"],train_1.loc[:,"category"]
sentences_dev, stance_dev, category_dev = dev_1.loc[:,"tokens"],dev_1.loc[:,"stance"],dev_1.loc[:,"category"]

In [6]:
# Get CBOW & Skip Gram Features for training and dev
X_train_cbow_w2v, X_train_sg_w2v = get_word_embeddings_features(sentences_train)
X_dev_cbow_w2v, X_dev_sg_w2v = get_word_embeddings_features(sentences_dev)

In [7]:
x_train = X_train_cbow_w2v + X_train_sg_w2v
x_dev = X_dev_cbow_w2v + X_dev_sg_w2v 

In [8]:
#Fitting The Stance Detection Model using Logistic Regression (W2v)
lr_w2v=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_w2v.fit(x_train, stance_train)  #model

#Predict y value for test dataset
stance_predict = lr_w2v.predict(x_dev)
stance_prob = lr_w2v.predict_proba(x_dev)[:,1]

#Fitting The Classification Model using Logistic Regression (W2v)
lr_w2v2=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_w2v2.fit(x_train, category_train)  #model

#Predict y value for test dataset
categ_predict = lr_w2v2.predict(x_dev)
categ_prob = lr_w2v2.predict_proba(x_dev)[:,1]

1000


In [9]:
print(metrics.classification_report(stance_predict,stance_dev))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.80      0.89      1000

    accuracy                           0.80      1000
   macro avg       0.33      0.27      0.30      1000
weighted avg       1.00      0.80      0.89      1000



/home/iten/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/iten/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/iten/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
print(metrics.classification_report(categ_predict,category_dev))

              precision    recall  f1-score   support

      advice       0.00      0.00      0.00         0
   celebrity       0.00      0.00      0.00         1
   info_news       0.00      0.00      0.00         0
      others       0.00      0.00      0.00         0
    personal       0.00      0.00      0.00         0
        plan       0.00      0.00      0.00         0
    requests       0.90      0.02      0.04       963
restrictions       0.00      0.00      0.00         0
      rumors       0.00      0.00      0.00         0
   unrelated       0.19      0.19      0.19        36

    accuracy                           0.03      1000
   macro avg       0.11      0.02      0.02      1000
weighted avg       0.87      0.03      0.04      1000



/home/iten/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/iten/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/iten/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
